# Modules

In [198]:
%pylab --no-import-all inline
#import EKPSolvers
import gurobipy as grb

Populating the interactive namespace from numpy and matplotlib


# Initialization

In [199]:
def sort_kp(c,w):
    t = [ (c[i],w[i],c[i]/w[i]) for i in range(len(c)) ]
    t.sort(key=lambda x : x[2])
    c = [p[0] for p in t]
    w = [p[1] for p in t]
    return (c,w)
    
cw = sort_kp([0.1,0.1,0.3,0.5],[2,1,3,2])

c = np.array(cw[0])
w = np.array(cw[1])
rhs = 4

c_dict = { i : c[i] for i in range(0,len(c)) }
w_dict = { i : w[i] for i in range(0,len(w)) }

vtype = grb.GRB.BINARY

# Optimization

In [200]:
model = grb.Model("knapsack")
vars = model.addVars(range(0,len(c_dict)),lb=0.0,ub=1.0, \
                     obj=c_dict,vtype=vtype)
model.addConstr(vars.prod(w_dict),grb.GRB.EQUAL,rhs)

fname = 'test.lp'
model.write(fname)

with open(fname) as f:
    content = ''
    for line in f.readlines():
        content = content + line
print(content)



\ Model knapsack
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  0.1 C0 + 0.3 C1 + 0.1 C2 + 0.5 C3
Subject To
 R0: 2 C0 + 3 C1 + C2 + 2 C3 = 4
Bounds
Binaries
 C0 C1 C2 C3
End



In [201]:
model.optimize()

Optimize a model with 1 rows, 4 columns and 4 nonzeros
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e-01, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
Presolve removed 1 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e-01, best bound 4.000000000000e-01, gap 0.0000%


# Solution

In [202]:
x = np.array([v.X for v in vars.values()])

print(c,w)
print('objective:','{:.20f}'.format(sum(x*c)))
print('constraint:',sum(x*w),'=',rhs)
print('solution:',x)

[0.1 0.3 0.1 0.5] [2 3 1 2]
objective: 0.40000000000000002220
constraint: 4.0 = 4
solution: [0. 1. 1. 0.]
